In [36]:
import io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# ---------------------------
# Helpers para Layout e Estilo
# ---------------------------
default_layout = widgets.Layout(width='300px')
default_style = {'description_width': 'initial'}

def format_text(text, bold, italic):
    """
    Formata o texto com MathText, aplicando negrito e/ou itálico.
    """
    if bold and italic:
        return fr'$\textbf{{\textit{{{text}}}}}$'
    elif bold:
        return fr'$\textbf{{{text}}}$'
    elif italic:
        return fr'$\textit{{{text}}}$'
    else:
        return text

# ---------------------------
# DataFrame Global
# ---------------------------
df = None

# ---------------------------
# 1. Widgets para Carregamento de Dados
# ---------------------------
upload_widget = widgets.FileUpload(
    accept='.csv, .xlsx',
    multiple=False,
    description='Carregar arquivo',
    layout=default_layout,
    style=default_style
)

dropdown_x = widgets.Dropdown(
    options=[],
    description='Eixo X:',
    disabled=True,
    layout=default_layout,
    style=default_style
)

# ---------------------------
# 2. Widgets para Tratamento de Dados
# ---------------------------
missing_checkbox = widgets.Checkbox(
    value=False,
    description='Remover valores faltantes',
    layout=default_layout,
    style=default_style
)
missing_method = widgets.Dropdown(
    options=['Remover linha', 'Substituir pela média', 'Substituir pelo valor anterior'],
    description='Tratamento faltantes:',
    layout=default_layout,
    style=default_style
)
normalize_checkbox = widgets.Checkbox(
    value=False,
    description='Normalizar dados',
    layout=default_layout,
    style=default_style
)
normalize_method = widgets.Dropdown(
    options=['Min-Max [0,1]', 'Z-score', 'Nenhuma'],
    description='Tipo normalização:',
    layout=default_layout,
    style=default_style
)
transform_dropdown = widgets.Dropdown(
    options=['Nenhuma', 'Log', 'Raiz Quadrada'],
    description='Transformação:',
    layout=default_layout,
    style=default_style
)

filter_x_text = widgets.Text(
    value='Nenhum',
    description='Filtrar eixo X (mín,máx):',
    placeholder='ex: 0,100 ou Nenhum',
    layout=default_layout,
    style=default_style
)
filter_y_text = widgets.Text(
    value='Nenhum',
    description='Filtrar eixo Y (mín,máx):',
    placeholder='ex: 0,100 ou Nenhum',
    layout=default_layout,
    style=default_style
)

moving_avg_checkbox = widgets.Checkbox(
    value=False,
    description='Aplicar média móvel',
    layout=default_layout,
    style=default_style
)
moving_avg_window = widgets.IntText(
    value=5,
    description='Tamanho da janela:',
    disabled=True,
    layout=default_layout,
    style=default_style
)
def on_moving_avg_change(change):
    moving_avg_window.disabled = not change['new']
moving_avg_checkbox.observe(on_moving_avg_change, names='value')

# ---------------------------
# 3. Configuração Visual Global (Título, Eixo X, Eixo Y, etc.)
# ---------------------------
title_text = widgets.Text(
    value='Meu Gráfico',
    description='Título:',
    layout=default_layout,
    style=default_style
)
title_bold_checkbox = widgets.Checkbox(
    value=False,
    description='Negrito',
    layout=widgets.Layout(width='100px'),
    style=default_style
)
title_italic_checkbox = widgets.Checkbox(
    value=False,
    description='Itálico',
    layout=widgets.Layout(width='100px'),
    style=default_style
)
title_style_box = widgets.HBox([title_bold_checkbox, title_italic_checkbox])

xlabel_text = widgets.Text(
    value='Eixo X',
    description='Rótulo X:',
    layout=default_layout,
    style=default_style
)
xlabel_bold_checkbox = widgets.Checkbox(
    value=False,
    description='Negrito',
    layout=widgets.Layout(width='100px'),
    style=default_style
)
xlabel_italic_checkbox = widgets.Checkbox(
    value=False,
    description='Itálico',
    layout=widgets.Layout(width='100px'),
    style=default_style
)
xlabel_style_box = widgets.HBox([xlabel_bold_checkbox, xlabel_italic_checkbox])

ylabel_text = widgets.Text(
    value='Eixo Y',
    description='Rótulo Y:',
    layout=default_layout,
    style=default_style
)
ylabel_bold_checkbox = widgets.Checkbox(
    value=False,
    description='Negrito',
    layout=widgets.Layout(width='100px'),
    style=default_style
)
ylabel_italic_checkbox = widgets.Checkbox(
    value=False,
    description='Itálico',
    layout=widgets.Layout(width='100px'),
    style=default_style
)
ylabel_style_box = widgets.HBox([ylabel_bold_checkbox, ylabel_italic_checkbox])

x_limits_text = widgets.Text(
    value='Nenhum',
    description='Limites Eixo X (min,máx):',
    placeholder='ex: 0,100 ou Nenhum',
    layout=default_layout,
    style=default_style
)
y_limits_text = widgets.Text(
    value='Nenhum',
    description='Limites Eixo Y (min,máx):',
    placeholder='ex: 0,1 ou Nenhum',
    layout=default_layout,
    style=default_style
)

grid_checkbox = widgets.Checkbox(
    value=False,
    description='Exibir grade',
    layout=default_layout,
    style=default_style
)
grid_axes = widgets.Dropdown(
    options=['X', 'Y', 'Ambos'],
    value='Ambos',
    description='Eixos da grade:',
    layout=default_layout,
    style=default_style
)

legend_checkbox = widgets.Checkbox(
    value=False,
    description='Exibir legenda',
    layout=default_layout,
    style=default_style
)
font_size = widgets.IntText(
    value=12,
    description='Tamanho da fonte:',
    layout=default_layout,
    style=default_style
)

# ---------------------------
# 4. Checkboxes para colunas de Y e Estilos Dinâmicos
# ---------------------------
checkboxes_y = {}  # col -> Checkbox
style_widgets_y = {}  # col -> dicionário de widgets de estilo
y_checkboxes_box = widgets.VBox([])  # para mostrar as checkboxes

# Container onde exibiremos os painéis de estilo de cada coluna selecionada
dynamic_y_style_box = widgets.VBox([])

def create_line_style_widget(col):
    """
    Cria um painel de widgets para personalizar a linha da coluna 'col'.
    (Cor, tipo de linha, espessura, marcadores, etc.)
    """
    header = widgets.HTML(f"<b>Estilo para '{col}'</b>")

    # Legenda customizável
    label_text = widgets.Text(
        value=col,
        description='Legenda:',
        layout=default_layout,
        style=default_style
    )
    label_bold = widgets.Checkbox(
        value=False,
        description='Negrito',
        layout=widgets.Layout(width='100px'),
        style=default_style
    )
    label_italic = widgets.Checkbox(
        value=False,
        description='Itálico',
        layout=widgets.Layout(width='100px'),
        style=default_style
    )
    label_style_box = widgets.HBox([label_bold, label_italic])

    # Linha
    line_color = widgets.Dropdown(
        options=['azul', 'vermelho', 'preto', 'verde', 'laranja', 'roxo'],
        value='azul',
        description='Cor da linha:',
        layout=default_layout,
        style=default_style
    )
    line_style = widgets.Dropdown(
        options=['contínua', 'tracejada', 'pontilhada', 'traço e ponto'],
        value='contínua',
        description='Tipo da linha:',
        layout=default_layout,
        style=default_style
    )
    line_width = widgets.IntText(
        value=2,
        description='Espessura da linha:',
        layout=default_layout,
        style=default_style
    )

    # Marcadores
    marker_checkbox = widgets.Checkbox(
        value=False,
        description='Exibir marcadores',
        layout=default_layout,
        style=default_style
    )
    marker_type = widgets.Dropdown(
        options=['círculo', 'quadrado', 'triângulo', 'losango', 'estrela'],
        value='círculo',
        description='Tipo marcador:',
        layout=default_layout,
        style=default_style
    )
    marker_color = widgets.Dropdown(
        options=['azul', 'vermelho', 'preto', 'verde', 'laranja', 'roxo'],
        value='preto',
        description='Cor marcador:',
        layout=default_layout,
        style=default_style
    )
    marker_size = widgets.IntText(
        value=5,
        description='Tamanho marcador:',
        layout=default_layout,
        style=default_style
    )

    box = widgets.VBox([
        header,
        label_text,
        label_style_box,
        line_color,
        line_style,
        line_width,
        marker_checkbox,
        marker_type,
        marker_color,
        marker_size
    ])

    return {
        'panel': box,
        'label_text': label_text,
        'label_bold': label_bold,
        'label_italic': label_italic,
        'line_color': line_color,
        'line_style': line_style,
        'line_width': line_width,
        'marker_checkbox': marker_checkbox,
        'marker_type': marker_type,
        'marker_color': marker_color,
        'marker_size': marker_size
    }

def on_y_checkbox_change(change):
    """
    Sempre que um checkbox de Y for marcado/desmarcado,
    atualizamos o painel de estilo correspondente.
    """
    if change['name'] == 'value':
        col = change['owner'].description  # nome da coluna
        checked = change['new']

        if checked:
            # Se foi marcado, cria os widgets de estilo (se não existirem)
            if col not in style_widgets_y:
                style_widgets_y[col] = create_line_style_widget(col)
        else:
            # Se foi desmarcado, remove os widgets de estilo
            if col in style_widgets_y:
                del style_widgets_y[col]

        # Atualiza o dynamic_y_style_box
        dynamic_y_style_box.children = [style_widgets_y[c]['panel'] for c in style_widgets_y]

def create_y_checkboxes(col_options):
    """
    Cria uma checkbox para cada coluna (exceto X),
    permitindo selecionar quais vão para o eixo Y.
    """
    checkboxes = []
    for col in col_options:
        cb = widgets.Checkbox(
            value=False,
            description=col,
            layout=default_layout,
            style=default_style
        )
        cb.observe(on_y_checkbox_change, names='value')
        checkboxes.append(cb)
        checkboxes_y[col] = cb

    y_checkboxes_box.children = checkboxes

# ---------------------------
# 5. Botão de Plotagem e Área de Saída
# ---------------------------
plot_button = widgets.Button(
    description='Plotar Gráfico',
    button_style='success',
    layout=widgets.Layout(width='150px')
)
output_area = widgets.Output()

save_checkbox = widgets.Checkbox(
    value=False,
    description='Salvar gráfico',
    layout=default_layout,
    style=default_style
)

export_format = widgets.Dropdown(
    options=['PNG', 'PDF', 'SVG', 'JPG'],
    value='PNG',
    description='Formato:',
    layout=default_layout,
    style=default_style
)
file_name_text = widgets.Text(
    value='grafico',
    description='Nome do arquivo:',
    layout=default_layout,
    style=default_style
)

# ---------------------------
# Atualiza as opções de seleção quando o arquivo é carregado
# ---------------------------
def on_upload_change(change):
    global df
    if upload_widget.value:
        uploaded_file = list(upload_widget.value.values())[0]
        name = uploaded_file['metadata']['name']
        content = uploaded_file['content']
        try:
            if name.endswith('.csv'):
                df = pd.read_csv(io.BytesIO(content))
            elif name.endswith('.xlsx'):
                df = pd.read_excel(io.BytesIO(content))
            else:
                with output_area:
                    clear_output()
                    print("Formato de arquivo não suportado.")
                return

            col_options = list(df.columns)

            # Atualiza dropdown X
            dropdown_x.options = col_options
            dropdown_x.disabled = False

            # Cria checkboxes para colunas de Y (exceto que o usuário pode marcar/desmarcar livremente)
            create_y_checkboxes(col_options)

            with output_area:
                clear_output()
                print(f"Arquivo '{name}' carregado com sucesso!")
        except Exception as e:
            with output_area:
                clear_output()
                print("Erro ao ler o arquivo:", e)

upload_widget.observe(on_upload_change, names='value')

# ---------------------------
# Função principal de plotagem
# ---------------------------
def plot_graph(b):
    with output_area:
        clear_output()
        if df is None:
            print("Por favor, carregue um arquivo primeiro.")
            return

        data = df.copy()

        # 1. Tratamento de valores faltantes
        if missing_checkbox.value:
            if missing_method.value == 'Remover linha':
                data = data.dropna()
            elif missing_method.value == 'Substituir pela média':
                data = data.fillna(data.mean(numeric_only=True))
            elif missing_method.value == 'Substituir pelo valor anterior':
                data = data.fillna(method='ffill')

        # 2. Normalização
        if normalize_checkbox.value and normalize_method.value != 'Nenhuma':
            cols_num = data.select_dtypes(include=[np.number]).columns
            if normalize_method.value == 'Min-Max [0,1]':
                data[cols_num] = (data[cols_num] - data[cols_num].min()) / (data[cols_num].max() - data[cols_num].min())
            elif normalize_method.value == 'Z-score':
                data[cols_num] = (data[cols_num] - data[cols_num].mean()) / data[cols_num].std()

        # 3. Transformação
        if transform_dropdown.value != 'Nenhuma':
            cols_num = data.select_dtypes(include=[np.number]).columns
            if transform_dropdown.value == 'Log':
                data[cols_num] = data[cols_num].applymap(lambda x: np.log(x) if x > 0 else np.nan)
            elif transform_dropdown.value == 'Raiz Quadrada':
                data[cols_num] = data[cols_num].applymap(lambda x: np.sqrt(x) if x >= 0 else np.nan)

        # 4. Filtros
        if filter_x_text.value.strip().lower() != 'nenhum':
            try:
                min_val, max_val = map(float, filter_x_text.value.split(','))
                data = data[(data[dropdown_x.value] >= min_val) & (data[dropdown_x.value] <= max_val)]
            except Exception as e:
                print("Erro no filtro do eixo X:", e)

        if filter_y_text.value.strip().lower() != 'nenhum':
            try:
                min_val, max_val = map(float, filter_y_text.value.split(','))
                # Para cada coluna marcada, aplica o filtro
                for col, cb in checkboxes_y.items():
                    if cb.value:  # se estiver marcado
                        data = data[(data[col] >= min_val) & (data[col] <= max_val)]
            except Exception as e:
                print("Erro no filtro do eixo Y:", e)

        # 5. Média móvel
        if moving_avg_checkbox.value:
            window = moving_avg_window.value
            for col, cb in checkboxes_y.items():
                if cb.value:
                    data[col] = data[col].rolling(window=window, min_periods=1).mean()

        # 6. Plotagem
        plt.figure(figsize=(8, 5))

        x_col = dropdown_x.value
        if x_col not in data.columns:
            print("Coluna de X inválida. Selecione novamente.")
            return

        x = data[x_col]

        ls_map = {
            'contínua': '-',
            'tracejada': '--',
            'pontilhada': ':',
            'traço e ponto': '-.'
        }
        color_map = {
            'azul': 'blue',
            'vermelho': 'red',
            'preto': 'black',
            'verde': 'green',
            'laranja': 'orange',
            'roxo': 'purple'
        }
        marker_map = {
            'círculo': 'o',
            'quadrado': 's',
            'triângulo': '^',
            'losango': 'D',
            'estrela': '*'
        }

        any_plotted = False

        for col, cb in checkboxes_y.items():
            if cb.value:  # se o usuário marcou essa coluna
                # Recupera os widgets de estilo
                style = style_widgets_y[col]

                # Formata a legenda
                label_text_formatted = format_text(
                    style['label_text'].value,
                    style['label_bold'].value,
                    style['label_italic'].value
                )

                line_col = color_map.get(style['line_color'].value, 'blue')
                line_ls = ls_map.get(style['line_style'].value, '-')
                lw = style['line_width'].value

                y = data[col]
                if style['marker_checkbox'].value:
                    mk = marker_map.get(style['marker_type'].value, 'o')
                    plt.plot(
                        x, y,
                        linestyle=line_ls,
                        color=line_col,
                        linewidth=lw,
                        marker=mk,
                        markersize=style['marker_size'].value,
                        label=label_text_formatted
                    )
                else:
                    plt.plot(
                        x, y,
                        linestyle=line_ls,
                        color=line_col,
                        linewidth=lw,
                        label=label_text_formatted
                    )
                any_plotted = True

        if not any_plotted:
            print("Nenhuma coluna de Y foi selecionada.")
            return

        # 7. Limites dos eixos (se definidos)
        if x_limits_text.value.strip().lower() != 'nenhum':
            try:
                xmin, xmax = map(float, x_limits_text.value.split(','))
                plt.xlim(xmin, xmax)
            except Exception as e:
                print("Erro nos limites do eixo X:", e)
        if y_limits_text.value.strip().lower() != 'nenhum':
            try:
                ymin, ymax = map(float, y_limits_text.value.split(','))
                plt.ylim(ymin, ymax)
            except Exception as e:
                print("Erro nos limites do eixo Y:", e)

        # 8. Título e rótulos com formatação
        title_str = format_text(title_text.value, title_bold_checkbox.value, title_italic_checkbox.value)
        plt.title(title_str, fontsize=font_size.value + 2)
        xlabel_str = format_text(xlabel_text.value, xlabel_bold_checkbox.value, xlabel_italic_checkbox.value)
        plt.xlabel(xlabel_str, fontsize=font_size.value)
        ylabel_str = format_text(ylabel_text.value, ylabel_bold_checkbox.value, ylabel_italic_checkbox.value)
        plt.ylabel(ylabel_str, fontsize=font_size.value)

        # 9. Grade
        if grid_checkbox.value:
            if grid_axes.value == 'X':
                plt.grid(axis='x')
            elif grid_axes.value == 'Y':
                plt.grid(axis='y')
            else:
                plt.grid(True)

        # 10. Legenda
        if legend_checkbox.value:
            plt.legend(fontsize=font_size.value)

        plt.show()

        # 11. Exportar gráfico se marcado
        # (se o usuário quer salvar em PNG, PDF etc.)
        # Exemplo:
        #  - file_name_text.value = 'grafico'
        #  - export_format.value = 'PNG'
        if save_checkbox.value:
            fmt = export_format.value.lower()
            filename = file_name_text.value + '.' + fmt
            plt.savefig(filename)
            print(f"Gráfico salvo como {filename}.")

plot_button.on_click(plot_graph)

# ---------------------------
# Layout da Interface
# ---------------------------
upload_box = widgets.VBox([
    widgets.HTML("<h3>1. Carregamento de Dados</h3>"),
    upload_widget
])

x_box = widgets.VBox([
    widgets.HTML("<h3>2. Selecione o Eixo X</h3>"),
    dropdown_x
])

y_selection_box = widgets.VBox([
    widgets.HTML("<h3>3. Selecione Colunas para Y</h3>"),
    widgets.HTML("Marque as colunas que deseja plotar:"),
    y_checkboxes_box,
    widgets.HTML("<b>Personalização de cada Y:</b>"),
    dynamic_y_style_box
])

treatment_box = widgets.VBox([
    widgets.HTML("<h3>4. Tratamento de Dados</h3>"),
    missing_checkbox,
    missing_method,
    normalize_checkbox,
    normalize_method,
    transform_dropdown,
    filter_x_text,
    filter_y_text,
    moving_avg_checkbox,
    moving_avg_window
])

visual_box = widgets.VBox([
    widgets.HTML("<h3>5. Configuração Visual Global</h3>"),
    widgets.HTML("<b>Título do Gráfico</b>"),
    title_text,
    title_style_box,
    widgets.HTML("<b>Rótulo Eixo X</b>"),
    xlabel_text,
    xlabel_style_box,
    widgets.HTML("<b>Rótulo Eixo Y</b>"),
    ylabel_text,
    ylabel_style_box,
    x_limits_text,
    y_limits_text,
    grid_checkbox,
    grid_axes,
    legend_checkbox,
    font_size
])

export_box = widgets.VBox([
    widgets.HTML("<h3>6. Exportação</h3>"),
    save_checkbox,
    export_format,
    file_name_text
])

ui = widgets.VBox([
    upload_box,
    x_box,
    y_selection_box,
    treatment_box,
    visual_box,
    plot_button,
    export_box,
    output_area
])

display(ui)
